## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be using the dataset provided in the assignment

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's read the dataset into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
predictors=concrete_data[['Cement','Blast Furnace Slag','Fly Ash','Water','Superplasticizer','Coarse Aggregate','Fine Aggregate']] #all colums except Strength and age

target=concrete_data[['Strength']] #strength as the target

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [8]:
target.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

7

<a id="item1"></a>

<a id="item1"></a>

## Import Keras

#### Let's go ahead and import the Keras library

In [11]:
import keras

Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [14]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

## Train and Test the Network

Let's call the function now to create our model.

In [16]:
# build the model
model = regression_model()

W0716 13:11:59.522371 11824 deprecation.py:506] From C:\Users\Saurabh\anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Next, we will train the model for 50 epochs.


In [17]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

W0716 13:12:31.734730 11824 module_wrapper.py:139] From C:\Users\Saurabh\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/50
 - 0s - loss: 1560.0860
Epoch 2/50
 - 0s - loss: 1543.3181
Epoch 3/50
 - 0s - loss: 1526.5548
Epoch 4/50
 - 0s - loss: 1510.2993
Epoch 5/50
 - 0s - loss: 1493.6419
Epoch 6/50
 - 0s - loss: 1476.0789
Epoch 7/50
 - 0s - loss: 1458.0478
Epoch 8/50
 - 0s - loss: 1439.0895
Epoch 9/50
 - 0s - loss: 1419.0255
Epoch 10/50
 - 0s - loss: 1398.7153
Epoch 11/50
 - 0s - loss: 1377.1515
Epoch 12/50
 - 0s - loss: 1353.9943
Epoch 13/50
 - 0s - loss: 1330.3391
Epoch 14/50
 - 0s - loss: 1304.8402
Epoch 15/50
 - 0s - loss: 1279.0662
Epoch 16/50
 - 0s - loss: 1251.5423
Epoch 17/50
 - 0s - loss: 1223.2121
Epoch 18/50
 - 0s - loss: 1193.6754
Epoch 19/50
 - 0s - loss: 1163.2045
Epoch 20/50
 - 0s - loss: 1131.6325
Epoch 21/50
 - 0s - loss: 1099.2505
Epoch 22/50
 - 0s - loss: 1065.8693
Epoch 23/50
 - 0s - loss: 1032.1646
Epoch 24/50
 - 0s - loss: 998.5379
Epoch 25/50
 - 0s - loss: 963.8174
Epoch 26/50
 - 0s - loss: 929.8807
Epoch 27/50
 - 0s - loss: 895.8828
Epoch 28/50
 - 0s - loss: 861.7321
Epoch 

Next we need to evaluate the model on the test data.

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 146us/step


326.6012402383255

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

326.601235787081 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [21]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 177.307373046875
MSE 2: 190.1198027280542
MSE 3: 169.42332769597618
MSE 4: 175.3579708210473
MSE 5: 166.45121414530237
MSE 6: 130.48933287191545
MSE 7: 163.50924094980974
MSE 8: 132.06045542102802
MSE 9: 150.2785458364147
MSE 10: 143.8162244778235
MSE 11: 135.87018883961304
MSE 12: 128.64742155599748
MSE 13: 130.00415058814977
MSE 14: 147.9113270781187
MSE 15: 126.10271192137091
MSE 16: 115.19435164071982
MSE 17: 128.29778168810995
MSE 18: 107.57261645369545
MSE 19: 121.55476804072804
MSE 20: 137.60263323243768
MSE 21: 120.20679333140549
MSE 22: 133.0996252263634
MSE 23: 133.61890561989597
MSE 24: 127.3406218000986
MSE 25: 115.04757804006435
MSE 26: 123.08799462179536
MSE 27: 142.31491661689043
MSE 28: 131.78929241884103
MSE 29: 136.3424986311533
MSE 30: 129.50709627515675
MSE 31: 123.72343237423202
MSE 32: 138.16314451594184
MSE 33: 126.79833722654669
MSE 34: 122.07624631715052
MSE 35: 133.7850210442898
MSE 36: 140.9267331219028
MSE 37: 127.33508241523816
MSE 38: 125.0522418469

As Compare to Part A Mean decresed and Standard Deviation increased.